# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 9:11AM,257207,12,U1218,"Uniderm USA, Inc",Released
1,Feb 13 2023 9:11AM,257207,12,U1219,"Uniderm USA, Inc",Released
2,Feb 13 2023 9:10AM,257205,10,MSP218431,"Methapharm, Inc.",Released
3,Feb 13 2023 9:10AM,257205,10,MSP218432,"Methapharm, Inc.",Released
4,Feb 13 2023 9:10AM,257205,10,MSP218434,"Methapharm, Inc.",Released
5,Feb 13 2023 9:10AM,257205,10,MSP218449,"Methapharm, Inc.",Released
6,Feb 13 2023 9:10AM,257205,10,MSP218453,"Methapharm, Inc.",Released
7,Feb 13 2023 9:10AM,257205,10,MSP218455,"Methapharm, Inc.",Released
8,Feb 13 2023 9:10AM,257205,10,MSP218458,"Methapharm, Inc.",Released
9,Feb 13 2023 9:10AM,257205,10,MSP218466,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,257201,Released,3
13,257202,Released,2
14,257203,Released,20
15,257205,Released,10
16,257207,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257201,NaN,3.0
257202,NaN,2.0
257203,NaN,20.0
257205,NaN,10.0
257207,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257179,0.0,1.0
257186,0.0,3.0
257191,0.0,12.0
257193,25.0,1.0
257194,0.0,41.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257179,0,1
257186,0,3
257191,0,12
257193,25,1
257194,0,41


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257179,0,1
1,257186,0,3
2,257191,0,12
3,257193,25,1
4,257194,0,41


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257179,,1
1,257186,,3
2,257191,,12
3,257193,25,1
4,257194,,41


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc"
2,Feb 13 2023 9:10AM,257205,10,"Methapharm, Inc."
12,Feb 13 2023 9:05AM,257203,10,ISDIN Corporation
32,Feb 13 2023 8:58AM,257202,10,"CLINUVEL, Inc."
34,Feb 13 2023 8:51AM,257200,10,Bio-PRF
44,Feb 13 2023 8:50AM,257201,10,Eye Pharma Inc
47,Feb 13 2023 8:50AM,257199,10,Bio-PRF
55,Feb 13 2023 8:47AM,257196,12,Hush Hush
105,Feb 13 2023 8:46AM,257198,19,"GCH Granules USA, Inc."
106,Feb 13 2023 8:43AM,257197,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",,2
1,Feb 13 2023 9:10AM,257205,10,"Methapharm, Inc.",,10
2,Feb 13 2023 9:05AM,257203,10,ISDIN Corporation,,20
3,Feb 13 2023 8:58AM,257202,10,"CLINUVEL, Inc.",,2
4,Feb 13 2023 8:51AM,257200,10,Bio-PRF,,10
5,Feb 13 2023 8:50AM,257201,10,Eye Pharma Inc,,3
6,Feb 13 2023 8:50AM,257199,10,Bio-PRF,,8
7,Feb 13 2023 8:47AM,257196,12,Hush Hush,,50
8,Feb 13 2023 8:46AM,257198,19,"GCH Granules USA, Inc.",,1
9,Feb 13 2023 8:43AM,257197,12,Hush Hush,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",2,
1,Feb 13 2023 9:10AM,257205,10,"Methapharm, Inc.",10,
2,Feb 13 2023 9:05AM,257203,10,ISDIN Corporation,20,
3,Feb 13 2023 8:58AM,257202,10,"CLINUVEL, Inc.",2,
4,Feb 13 2023 8:51AM,257200,10,Bio-PRF,10,
5,Feb 13 2023 8:50AM,257201,10,Eye Pharma Inc,3,
6,Feb 13 2023 8:50AM,257199,10,Bio-PRF,8,
7,Feb 13 2023 8:47AM,257196,12,Hush Hush,50,
8,Feb 13 2023 8:46AM,257198,19,"GCH Granules USA, Inc.",1,
9,Feb 13 2023 8:43AM,257197,12,Hush Hush,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",2,
1,Feb 13 2023 9:10AM,257205,10,"Methapharm, Inc.",10,
2,Feb 13 2023 9:05AM,257203,10,ISDIN Corporation,20,
3,Feb 13 2023 8:58AM,257202,10,"CLINUVEL, Inc.",2,
4,Feb 13 2023 8:51AM,257200,10,Bio-PRF,10,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",2,NaN
1,Feb 13 2023 9:10AM,257205,10,"Methapharm, Inc.",10,NaN
2,Feb 13 2023 9:05AM,257203,10,ISDIN Corporation,20,NaN
3,Feb 13 2023 8:58AM,257202,10,"CLINUVEL, Inc.",2,NaN
4,Feb 13 2023 8:51AM,257200,10,Bio-PRF,10,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",2,0.0
1,Feb 13 2023 9:10AM,257205,10,"Methapharm, Inc.",10,0.0
2,Feb 13 2023 9:05AM,257203,10,ISDIN Corporation,20,0.0
3,Feb 13 2023 8:58AM,257202,10,"CLINUVEL, Inc.",2,0.0
4,Feb 13 2023 8:51AM,257200,10,Bio-PRF,10,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2057575,57,0.0
12,771600,53,0.0
19,1285971,97,25.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2057575,57,0.0
1,12,771600,53,0.0
2,19,1285971,97,25.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,57,0.0
1,12,53,0.0
2,19,97,25.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,12,Released,53.0
2,19,Released,97.0
3,10,Executing,0.0
4,12,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19
Status,,,
Executing,0.0,0.0,25.0
Released,57.0,53.0,97.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19
0,Executing,0.0,0.0,25.0
1,Released,57.0,53.0,97.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19
0,Executing,0.0,0.0,25.0
1,Released,57.0,53.0,97.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()